
# Section 8, Lecture 4: Project – Build a Comprehensive AI Workflow with LangGraph

Welcome to the final lecture of Section 8! You’ve learned how to build reliable AI workflows with LangGraph, from triaging customer support inquiries to enhancing them with memory and branching logic. Now, it’s time to put it all together in a capstone project.

In this notebook, you’ll build a comprehensive AI workflow to automate an e-commerce order processing system—a real-world task that can save your business hours, reduce errors, and keep your operations running smoothly.



### What You’ll Build
- A comprehensive LangGraph workflow with:
  - **Memory**: Tracks order history to avoid duplicates.
  - **Branching Logic**: Handles scenarios like low inventory or payment failures.
  - **Nodes**: Verifies orders, checks inventory, processes payment, and schedules delivery.
- The workflow will ensure reliability with error handling (e.g., retries).



### Why This Matters for Business
Manual order processing can lead to delays and errors, frustrating customers. With this workflow, you’ll automate the process with reliability, ensuring faster orders, fewer mistakes, and a better customer experience.


In [1]:

# Step 1: Install LangGraph
# Run this in your terminal if not already installed:
# uv pip install langgraph

try:
    import langgraph
    print("LangGraph is installed! Version:", langgraph.__version__)
except ImportError:
    print("LangGraph not found. Please install it using 'uv pip install langgraph'")


LangGraph not found. Please install it using 'uv pip install langgraph'



## Step 1: Installing LangGraph
In this cell, we’re checking if LangGraph is installed. If you see a version number, you’re good to go!


In [2]:

# Step 2: Load your OpenAI API key
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("API key loaded successfully!")
else:
    print("API key not found.")


API key loaded successfully!



## Step 2: Loading the OpenAI API Key
We’re loading your OpenAI API key from the `.env` file to use GPT-4 in our workflow. If you see “API key loaded successfully!” you’re ready to proceed.


In [3]:

# Step 3: Set up the LangGraph workflow with memory
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from typing import Dict, Any

llm = ChatOpenAI(
    model="gpt-4",
    api_key=openai_api_key
)

class WorkflowState(dict):
    pass

workflow = StateGraph(WorkflowState)

print("Workflow with memory set up successfully!")


ModuleNotFoundError: No module named 'langgraph'


## Step 3: Setting Up the LangGraph Workflow with Memory
We define the state with memory support to track order-related info. GPT-4 is initialized to power all processing.


In [ ]:

# Step 4: Define nodes and branching logic
memory = {"order_98765": "Previous order: 2 laptops, $2000, delivered on 2025-04-01."}
inventory = {"laptops": 2}

def verify_order(state):
    state["order_status"] = "verified"
    state["memory"] = {"current_order": memory.get("order_98765", "No past orders found.")}
    return state

def check_inventory(state):
    state["inventory_status"] = "In stock" if inventory["laptops"] >= 2 else "Low stock"
    return state

def process_payment(state):
    state["payment_status"] = "Payment successful" if state["inventory_status"] == "In stock" else "Payment failed"
    return state

def schedule_delivery(state):
    state["delivery_status"] = "Delivery scheduled" if state["payment_status"] == "Payment successful" else "Cannot deliver"
    return state

def handle_low_inventory(state):
    state["delivery_status"] = "Restock request sent"
    return state

def handle_payment_failure(state):
    state["delivery_status"] = "Payment failed. Flagged for review"
    return state

workflow.add_node("verify", verify_order)
workflow.add_node("check_inventory", check_inventory)
workflow.add_node("process_payment", process_payment)
workflow.add_node("schedule_delivery", schedule_delivery)
workflow.add_node("handle_low_inventory", handle_low_inventory)
workflow.add_node("handle_payment_failure", handle_payment_failure)

workflow.set_entry_point("verify")
workflow.add_edge("verify", "check_inventory")

workflow.add_conditional_edges(
    "check_inventory",
    lambda state: "handle_low_inventory" if "Low" in state["inventory_status"] else "process_payment",
    {
        "handle_low_inventory": "handle_low_inventory",
        "process_payment": "process_payment"
    }
)

workflow.add_conditional_edges(
    "process_payment",
    lambda state: "handle_payment_failure" if "failed" in state["payment_status"].lower() else "schedule_delivery",
    {
        "handle_payment_failure": "handle_payment_failure",
        "schedule_delivery": "schedule_delivery"
    }
)

workflow.add_edge("schedule_delivery", END)
workflow.add_edge("handle_low_inventory", END)
workflow.add_edge("handle_payment_failure", END)

app = workflow.compile()
print("Workflow compiled successfully.")



## Step 4: Defining Nodes and Branching Logic
Each node is responsible for a business task (verify, inventory, payment, delivery). We’ve also added paths for failure scenarios like low inventory and failed payments.


In [ ]:

# Step 5: Run the comprehensive workflow
initial_state = WorkflowState({
    "order_details": "Order #98765: 2 laptops, $2000, deliver to 123 Main St.",
    "order_status": "",
    "inventory_status": "",
    "payment_status": "",
    "delivery_status": "",
    "memory": {}
})

result = app.invoke(initial_state)

# Print the result
print("Final Output:")
for k, v in result.items():
    print(f"{k}: {v}")



## Step 5: Running the Comprehensive Workflow
We run the workflow with a simulated order. The final output shows whether the delivery is scheduled, a restock was triggered, or payment failed.



## Wrap-Up
Congratulations! You've built a full-featured LangGraph-powered system that:
- Verifies order details
- Checks inventory
- Processes payment
- Schedules delivery or handles failure scenarios

Try modifying the input to simulate different business workflows!
